In [1]:
import numpy as np
from thirdai import distributed_bolt
from thirdai import bolt

In [2]:
from utils import (
    gen_single_sparse_layer_network,
    gen_training_data,
    get_categorical_acc,train_network,train_single_node_distributed_network
)

In [3]:
def build_sparse_hidden_layer_classifier(input_dim, sparse_dim, output_dim, sparsity):
    layers = [
        bolt.FullyConnected(
            dim=sparse_dim,
            sparsity=sparsity,
            activation_function="ReLU",
        ),
        bolt.FullyConnected(dim=output_dim, activation_function="Softmax"),
    ]
    network = bolt.DistributedNetwork(layers=layers, input_dim=input_dim)
    return network

In [4]:
input_dim = 10
hidden_dim = 10
output_dim = 10
network = build_sparse_hidden_layer_classifier(
    input_dim=input_dim, sparse_dim=hidden_dim, output_dim=output_dim, sparsity=1.0
)

Initializing Bolt network...
========= Bolt Network =========
InputLayer (Layer 0): dim=10
FullyConnectedLayer (Layer 1): dim=10, sparsity=1, act_func=ReLU
FullyConnectedLayer (Layer 2): dim=10, sparsity=1, act_func=Softmax
Initialized Network in 0 seconds


In [24]:
examples, labels = gen_training_data(n_classes=10, n_samples=1000)
train_single_node_distributed_network(network, examples, labels, epochs=10)

Distributed Network initialization done on this Node


In [25]:
x=network.get_indexed_sketch(layer_index=0,compression_density=0.5,is_set_biases=True,seed=2)
print(x)

(array([6, 7, 5, 1, 8], dtype=int32), array([-4.173293  , -2.7389598 , -0.41703063,  0.        , -0.64264446],
      dtype=float32))
the threshold is -0.398358
done calculating the sketch


In [26]:
weights=network.get_biases_gradients(0)
print(weights)

[ 1.0735991   0.         -0.39835846 -0.01555806  0.28673863 -0.41703063
 -4.173293   -2.7389598  -0.64264446  0.7378569 ]


In [27]:
network.set_gradients(layer_index=0,indices=x[0],values=x[1],is_set_biases=True)

inside the set gradients from tuple function
setting the bias gradients for the layer: 0
i am here
print bias gradient before assinging to 0
1.0736 0 -0.398358 -0.0155581 0.286739 -0.417031 -4.17329 -2.73896 -0.642644 0.737857 
print bias gradient after assinging to 0
0 0 0 0 0 0 0 0 0 0 
setting the bias gradients for the layer


In [28]:
weights_new=network.get_biases_gradients(0)
print(weights_new)

[ 0.          0.          0.          0.          0.         -0.41703063
 -4.173293   -2.7389598  -0.64264446  0.        ]
